# Part 3: Prognostics for Turbofan Engines 🩺

Prognostics is the prediction of Remaining Useful Life (RUL) until failure of a component based on the knowledge about current and future operation conditions (obtained through various sensors or physical models).

## 0. Introduction

The new C-MAPSS dataset DS02 from NASA provides degradation trajectories of 9 turbofan engines with unknown and different initial health condition for complete flights and two failure modes (HPT efficiency degradation & HPT efficiency degradation combined with LPT efficiency and capacity degradation). The data were synthetically generated with the Commercial Modular Aero-Propulsion System Simulation (C-MAPSS) dynamical model. The data contains multivariate sensors readings of the complete run-to-failure trajectories. Therefore, the records stop at the cycle/time the engine failed. A total number of 6.5M time stamps are available. Dataset copyright (c) by Manuel Arias.

**For training simplicity, the dataset has been preprocessed. The dataset has been downsampled from 1Hz to 0.1 Hz with an IIR 8th Order Chebyshev filter. Data format has been converted from double to float precison.**

In [ ]:
%matplotlib inline

import os
import time 
import random

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
import torch
import torch.nn as nn
from torchinfo import summary
from torch.utils.data import DataLoader, Dataset, SubsetRandomSampler
from tqdm import tqdm

if torch.cuda.is_available():
    device = torch.device("cuda")
elif torch.backends.mps.is_available():
    device = torch.device("mps")
else:
    device = torch.device("cpu")


In [ ]:
folder = os.getcwd()
filename = "../datasets/N-CMAPSS_DS02.csv"
print(filename)


## 1. Data exploration 🔍

In [ ]:
df = pd.read_csv(filename)
df.head()


In [ ]:
df.describe()


## 2. Feature Overview

In [ ]:
LABELS = ["RUL"]


Operating Conditions ($w$)

DASHlink- Flight Data For Tail 687.(2012). Retrieved on 2019-01-29 from https://c3.nasa.gov/dashlink/

In [ ]:
W_VAR = ["alt", "Mach", "TRA", "T2"]


Sensor readings ($X_s$)

In [ ]:
XS_VAR = [
    "T24",
    "T30",
    "T48",
    "T50",
    "P15",
    "P2",
    "P21",
    "P24",
    "Ps30",
    "P40",
    "P50",
    "Nf",
    "Nc",
    "Wf",
]


## 3. Developing your first prognostics model 💻
Step by step, you will learn how to build your first prognostics model from scratch!

### 3.1 Define Sequence Dataset 


In [ ]:
class SlidingWindowDataset(Dataset):
    def __init__(self, dataframe, window=50, stride=1, horizon=1, device="cpu"):
        """Sliding window dataset with RUL label

        Args:
            dataframe (pd.DataFrame): dataframe containing scenario descriptors and sensor reading
            window (int, optional): sequence window length. Defaults to 50.
            stride (int, optional): data stride length. Defaults to 1.
            horizon (int, optional): prediction forcasting length. Defaults to 1.
        """
        self.window = window
        self.stride = stride
        self.horizon = horizon

        self.X = np.array(dataframe[XS_VAR + W_VAR].values).astype(np.float32)
        self.y = np.array(dataframe["RUL"].values).astype(np.float32)
        if "ds" in dataframe.columns:
            unqiue_cycles = dataframe[["ds", "unit", "cycle"]].value_counts(sort=False)
        else:
            unqiue_cycles = dataframe[["unit", "cycle"]].value_counts(sort=False)
        self.indices = torch.from_numpy(self._get_indices(unqiue_cycles)).to(device)

    # TODO add comment
    def _get_indices(self, unqiue_cycles):
        cycles = unqiue_cycles.to_numpy()
        idx_list = []
        for i, c_count in enumerate(cycles):
            c_start = sum(cycles[:i])
            c_end = c_start + (c_count - self.window - self.horizon)
            if c_end + self.horizon < len(
                self.X
            ):  # handling y not in the last seq case
                idx_list += [_ for _ in np.arange(c_start, c_end + 1, self.stride)]
        return np.asarray([(idx, idx + self.window) for idx in idx_list])

    def __len__(self):
        return len(self.indices)

    def __getitem__(self, i):
        i_start, i_stop = self.indices[i]
        x = self.X[i_start:i_stop, :]
        y = self.y[i_start]
        return x, y


In [ ]:
def create_datasets(df, window_size, train_units, test_units, device="cpu"):
    df_train = df[df["unit"].isin(train_units)]
    train_dataset = SlidingWindowDataset(df_train, window=window_size)

    df_test = df[df["unit"].isin(test_units)]
    test_dataset = SlidingWindowDataset(df_test, window=window_size)

    # normalizing features
    scaler = MinMaxScaler()
    train_dataset.X = scaler.fit_transform(train_dataset.X)
    test_dataset.X = scaler.transform(test_dataset.X)

    # convert numpy array to tensors
    datasets = [train_dataset, test_dataset]
    for d in datasets:
        d.X = torch.from_numpy(d.X).to(device)
        d.y = torch.from_numpy(d.y).to(device)

    return datasets


def create_data_loaders(datasets, batch_size=256, val_split=0.2):
    d_train, d_test = datasets
    dataset_size = len(d_train)
    indices = list(range(dataset_size))
    split = int(np.floor(val_split * dataset_size))
    np.random.shuffle(indices)
    train_indices, val_indices = indices[split:], indices[:split]
    train_sampler = SubsetRandomSampler(train_indices)
    valid_sampler = SubsetRandomSampler(val_indices)

    train_loader = DataLoader(d_train, batch_size=batch_size, sampler=train_sampler)
    val_loader = DataLoader(d_train, batch_size=batch_size, sampler=valid_sampler)
    test_loader = DataLoader(d_test, batch_size=batch_size, shuffle=False)

    d_info = f"train_size: {len(train_indices)}\t"
    d_info += f"validation_size: {len(val_indices)}\t"
    d_info += f"test_size: {len(d_test)}"
    print(d_info)
    return train_loader, val_loader, test_loader


### 3.2 Define Trainer Class

In [ ]:
class Trainer:
    def __init__(
        self,
        model,
        optimizer,
        n_epochs=20,
        criterion=nn.MSELoss(),
        model_name="best_model",
        device="cpu",
    ):
        self.model = model
        self.optimizer = optimizer
        self.device = device
        self.n_epochs = n_epochs
        self.criterion = criterion

        # adding time_stamp to model name to make sure the save models don't overwrite each other,
        # you can customize your own model name with hyperparameters so that you can reload the model more easily
        time_stamp = time.strftime("%m%d%H%M%S")
        self.model_path = f"models/{model_name}_{time_stamp}.pt"

        self.losses = {split: [] for split in ["train", "eval", "test"]}

    def compute_loss(self, x, y, model=None):
        y = y.view(-1)
        y_pred = self.model(x)
        y_pred = y_pred.view(-1)
        loss = self.criterion(y, y_pred)
        return loss, y_pred, y

    def train_epoch(self, loader):
        self.model.train()
        # batch losses
        b_losses = []
        for x, y in loader:
            # Setting the optimizer gradient to Zero
            self.optimizer.zero_grad()
            x.to(torch.device(self.device))
            y.to(torch.device(self.device))

            loss, pred, target = self.compute_loss(x, y)

            # Backpropagate the training loss
            loss.backward()
            self.optimizer.step()
            b_losses.append(loss.cpu().detach().numpy())

        # aggregated losses across batches
        agg_loss = np.sqrt((np.asarray(b_losses) ** 2).mean())
        self.losses["train"].append(agg_loss)
        return agg_loss

    # decorator, equivalent to with torch.no_grad():
    @torch.no_grad()
    def eval_epoch(self, loader, split="eval"):
        self.model.eval()
        # batch losses
        b_losses = []
        for x, y in loader:
            x.to(torch.device(self.device))
            y.to(torch.device(self.device))

            loss, pred, target = self.compute_loss(x, y)

            b_losses.append(loss.cpu().detach().numpy())

        # aggregated losses across batches
        agg_loss = np.sqrt((np.asarray(b_losses) ** 2).mean())
        self.losses[split].append(agg_loss)
        return agg_loss

    def fit(self, loaders):
        print(f"Training model for {self.n_epochs} epochs...")
        train_loader, eval_loader, test_loader = loaders
        train_start = time.time()

        start_epoch = 0
        best_eval_loss = np.inf

        for epoch in range(start_epoch, self.n_epochs):
            epoch_start = time.time()

            train_loss = self.train_epoch(train_loader)
            eval_loss = self.eval_epoch(eval_loader, split="eval")
            test_loss = self.eval_epoch(test_loader, split="test")

            if eval_loss < best_eval_loss:
                best_eval_loss = eval_loss
                self.save(self.model, self.model_path)

            s = (
                f"[Epoch {epoch + 1}] "
                f"train_loss = {train_loss:.5f}, "
                f"eval_loss = {eval_loss:.5f}, "
                f"test_loss = {test_loss:.5f}"
            )

            epoch_time = time.time() - epoch_start
            s += f" [{epoch_time:.1f}s]"
            print(s)

        train_time = int(time.time() - train_start)

        print(f"Task done in {train_time}s")

    # decorator, equivalent to with torch.no_grad():
    @torch.no_grad()
    def eval_rul_prediction(self, test_loader):
        print(f"Evaluating test RUL...")

        ## MASK OUT EVAL and add explanation
        best_model = self.load(self.model)
        best_model.eval()

        preds = []
        trues = []

        for x, y in tqdm(test_loader):
            x = x.to(self.device)
            y = y.to(self.device)

            _, y_pred, y_target = self.compute_loss(x, y)
            preds.append(y_pred.detach().cpu().numpy())
            trues.append(y_target.detach().cpu().numpy())

        preds = np.concatenate(preds, axis=0)
        trues = np.concatenate(trues, axis=0)

        df = pd.DataFrame(
            {"pred": preds, "true": trues, "err": np.sqrt((preds - trues) ** 2)}
        )
        return df

    def save(self, model, model_path=None):
        os.makedirs(f"{folder}/models", exist_ok=True)
        if model_path is None:
            model_path = self.model_path
        torch.save(model.state_dict(), model_path)

    def load(self, model, model_path=None):
        """
        loads the prediction model's parameters
        """
        if model_path is None:
            model_path = self.model_path
        model.load_state_dict(torch.load(model_path, map_location=self.device))
        print(
            f"Model {model.__class__.__name__} saved in {model_path} loaded to {self.device}"
        )
        return model

    def plot_losses(self):
        """
        :param losses: dict with losses
        """
        linestyles = {
            "train": "solid",
            "eval": "dashed",
            "test": "dotted",
        }
        for split, loss in self.losses.items():
            ls = linestyles[split]
            plt.plot(range(1, 1 + len(loss)), loss, label=f"{split} loss", linestyle=ls)
            plt.yscale("log")

        plt.title("Training/Validation Losses")
        plt.xlabel("Epoch")
        plt.ylabel("Loss")
        plt.legend()
        plt.show()


### 3.3 Define Model: 1DCNN

Conventional CNN's developed for image tasks learn to extract features from the 2D input data. They are autonomous (require no domain expertise or prior info about the image) and thus can be applied to any image regardless of its dimensions. This is due to the fact that these CNN's go through an image by downsampling the image which we call straddling or windowing.  

Similarly 1D CNN learns to extract features from a time series data, by windowing over the data, considering a set of data observations each time. The benefit of using the CNN for sequence classification is that it can learn from the raw time series data, and in turn do not require domain expertise to engineer relevant features.

In [ ]:
def init_weights(m):
    if isinstance(m, nn.BatchNorm1d):
        m.weight.data.fill_(1.0)
        m.bias.data.zero_()
    elif isinstance(m, nn.Conv1d) or isinstance(m, nn.Linear):
        m.weight.data = nn.init.xavier_uniform_(
            m.weight.data, gain=nn.init.calculate_gain("relu")
        )
        if m.bias is not None:
            m.bias.data.zero_()


def Single1DCNNLayer(
    in_channels, out_channels=1, dropout=0.2, kernel_size=10, padding="same"
):
    # mask out batchnorm1d
    layer = [
        nn.Conv1d(in_channels, out_channels, kernel_size, padding=padding),
        nn.BatchNorm1d(out_channels),
    ]
    if dropout > 0.0:
        layer.append(nn.Dropout(dropout))
    layer.append(nn.ReLU())
    return layer


class CNN(nn.Module):
    """
    A 1D-CNN model that is based on the paper "Fusing physics-based and deep learning models for prognostics"
    from Manuel Arias Chao et al. (with batchnorm layers)
    """

    def __init__(
        self,
        in_channels=18,
        out_channels=1,
        window=50,
        n_ch=10,
        n_k=10,
        n_hidden=50,
        n_layers=3,
        dropout=0.0,
        padding="same",
    ):
        """
        Args:
            n_features (int, optional): number of input features. Defaults to 18.
            window (int, optional): sequence length. Defaults to 50.
            n_ch (int, optional): number of channels (filter size). Defaults to 10.
            n_k (int, optional): kernel size. Defaults to 10.
            n_hidden (int, optional): number of hidden neurons for regressor. Defaults to 50.
            n_layers (int, optional): number of convolution layers. Defaults to 5.
        """
        super().__init__()

        layers = []
        for i in range(n_layers):
            dim_in = in_channels if i == 0 else n_ch
            dim_out = n_ch if i != n_layers - 1 else 1
            layers += Single1DCNNLayer(
                dim_in, dim_out, dropout=dropout, kernel_size=n_k, padding=padding
            )

        self.cnn_layers = nn.Sequential(*layers)

        # Regression Output
        post_layers = [nn.Linear(in_features=window, out_features=n_hidden)]
        post_layers += [
            nn.ReLU(),
            nn.Linear(in_features=n_hidden, out_features=out_channels),
        ]
        self.rul_regressor = nn.Sequential(*post_layers)
        # Recursive apply weight initialization for all models
        self.apply(init_weights)

    def forward(self, x):

        # Change input shape from (b, window_size, in_channels) to (b, in_channels, window_size)
        x = x.permute(0, 2, 1)

        # Propagate input through Conv-Layers
        feature = self.cnn_layers(x)

        # Flatten the Feature Map
        feature = torch.flatten(feature, start_dim=1)
        output = self.rul_regressor(feature)

        return output


### 3.4 Training a model instance

In [ ]:
def seed_everything(seed: int):
    r"""Sets the seed for generating random numbers in PyTorch, numpy and
    Python.

    Args:
        seed (int): The desired seed.
    """
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)


In [ ]:
# dataset parameters
train_units = [16, 18, 20]
test_units = [15]
window_size = 50

# training parameters
batch_size = 256
base_lr = 5e-3
weight_decay = 1e-5
max_epochs = 10

# CNN model parameters
in_channels = 18
out_channels = 1
n_ch = 10
n_k = 9
n_hidden = 50
n_layers = 3
dropout = 0.0


In [ ]:
SEED = 271828  # choose your magic number
seed_everything(SEED)


In [ ]:
datasets = create_datasets(
    df, window_size=50, train_units=train_units, test_units=test_units, device=device
)
loaders = create_data_loaders(datasets, batch_size=batch_size)

model = CNN(
    in_channels=in_channels,
    out_channels=out_channels,
    n_ch=n_ch,
    n_k=n_k,
    n_hidden=n_hidden,
    n_layers=n_layers,
    dropout=dropout,
).to(device)
print(summary(model))

optimizer = torch.optim.Adam(model.parameters(), lr=base_lr, weight_decay=weight_decay)

criterion = nn.MSELoss()


In [ ]:
trainer = Trainer(
    model, optimizer, criterion=criterion, n_epochs=max_epochs, device=device
)

trainer.fit(loaders)


In [ ]:
trainer.plot_losses()


In [ ]:
# torch.save(model.state_dict(), "./models/cmapss_baseline.pt")
# model.load_state_dict(torch.load("./models/cmapss_baseline.pt", map_location=device))


### 3.5 Evaluation on the test set

In [ ]:
df_test = trainer.eval_rul_prediction(loaders[2])


In [ ]:
df_test.plot(y=["true", "pred"])


### 3.6 Introduce NASA score

In [ ]:
def nasa_score(y_pred, y_true):
    """
    Nasa score (in 1E5) as introduced in https://arxiv.org/abs/2003.00732
    """
    d = y_pred - y_true
    score = np.sum(np.exp(d[d >= 0] / 10) - 1) + np.sum(np.exp(-d[d < 0] / 13) - 1)
    return score / 1e5


In [ ]:
nasa_score(df_test["pred"], df_test["true"])


In [ ]:
rmse = np.sqrt(np.mean((df_test["pred"] - df_test["true"]) ** 2))
print(rmse)
